In [1]:
import numpy as np

def gini(classes, total):
    return 1 - sum((freq / total) ** 2 for freq in classes)


In [2]:
import pandas as pd
import math
from collections import Counter
import treelib

def CART(df, coluna_classe):
    total = len(df)
    if total == 0:
        return [0, None, None]  # ganho, coluna, valor_split

    classe_counts = df[coluna_classe].value_counts()
    gini_total = gini(classe_counts, total)

    melhor_ganho = [0, None, None]  # ganho, coluna, valor_split

    for coluna in df.columns:
        if coluna == coluna_classe:
            pass
        else:
            valores_unicos = df[coluna].unique()

            for valor in valores_unicos:
                df_esq = df[df[coluna] == valor]
                df_dir = df[df[coluna] != valor]

                total_esq = len(df_esq)
                total_dir = len(df_dir)

                if total_esq > 0 and total_dir > 0:
                    gini_esq = gini(df_esq[coluna_classe].value_counts(), total_esq)
                    gini_dir = gini(df_dir[coluna_classe].value_counts(), total_dir)

                    gini_split = (total_esq / total) * gini_esq + (total_dir / total) * gini_dir
                    ganho = gini_total - gini_split

                    if ganho > melhor_ganho[0]:
                        melhor_ganho = [ganho, coluna, valor]

    return melhor_ganho 

In [3]:
import pandas as pd
import math
from collections import Counter
import treelib

def geraArvore(df, coluna_classe, tree, parent=None, ramo_valor=None):
    melhorDivisao = CART(df, coluna_classe)

    if melhorDivisao[0] != 0:
        atributo = melhorDivisao[1]
        valor_split = melhorDivisao[2]
        node_id = f"{atributo}_{valor_split}_{len(tree.nodes)}"

        tag_node = f"{ramo_valor} → {atributo} == {valor_split}" if parent else f"{atributo} == {valor_split}"
        tree.create_node(tag=tag_node, identifier=node_id, parent=parent)

        # Ramo ESQUERDO (atributo == valor_split)
        df_esq = df[df[atributo] == valor_split]
        classes_esq = df_esq[coluna_classe].unique()

        if len(classes_esq) == 1:
            classe_final = str(classes_esq[0])
            folha_id = f"esq_{classe_final}_{len(tree.nodes)}"
            tree.create_node(tag=f"Sim → {classe_final}", identifier=folha_id, parent=node_id)
        else:
            geraArvore(df_esq.drop(columns=[atributo]), coluna_classe, tree, parent=node_id, ramo_valor="Sim")

        # Ramo DIREITO (atributo != valor_split)
        df_dir = df[df[atributo] != valor_split]
        classes_dir = df_dir[coluna_classe].unique()

        if len(classes_dir) == 1:
            classe_final = str(classes_dir[0])
            folha_id = f"dir_{classe_final}_{len(tree.nodes)}"
            tree.create_node(tag=f"Não → {classe_final}", identifier=folha_id, parent=node_id)
        else:
            geraArvore(df_dir.drop(columns=[atributo]), coluna_classe, tree, parent=node_id, ramo_valor="Não")

    else:
        classe_majoritaria = str(df[coluna_classe].mode()[0])
        folha_id = f"{classe_majoritaria}_{len(tree.nodes)}"
        tag = f"{ramo_valor} → {classe_majoritaria}" if ramo_valor else classe_majoritaria
        tree.create_node(tag=tag, identifier=folha_id, parent=parent)

In [4]:
import pandas as pd
import math
from collections import Counter
from treelib import Tree


# Abrindo o arquivo CSV
df = pd.read_csv('train.csv',sep = ",")

# Remover colunas que não são úteis para classificação
df.drop(columns=["PassengerId", "Name", "Ticket", "Cabin"], inplace=True)

coluna_classe = "Survived"
# TEste de abertura
#print(df.head()) 
  
#colocar na árvore
tree = Tree()
geraArvore(df,coluna_classe, tree)
tree.show()

Sex == male
├── Não → Pclass == 3
│   ├── Não → Fare == 151.55
│   │   ├── Não → Age == 57.0
│   │   │   ├── Não → Parch == 0
│   │   │   │   ├── Não → SibSp == 1
│   │   │   │   │   ├── Não → 1
│   │   │   │   │   └── Sim → Embarked == C
│   │   │   │   │       ├── Não → 1
│   │   │   │   │       └── Sim → 1
│   │   │   │   └── Sim → Embarked == S
│   │   │   │       ├── Não → SibSp == 1
│   │   │   │       │   ├── Não → 1
│   │   │   │       │   └── Sim → 1
│   │   │   │       └── Sim → SibSp == 1
│   │   │   │           ├── Não → 1
│   │   │   │           └── Sim → 1
│   │   │   └── Sim → 0
│   │   └── Sim → Age == 22.0
│   │       ├── Não → 0
│   │       └── Sim → 1
│   └── Sim → Embarked == S
│       ├── Não → Fare == 14.4542
│       │   ├── Não → Age == 17.0
│       │   │   ├── Não → Parch == 5
│       │   │   │   ├── Não → SibSp == 2
│       │   │   │   │   ├── Não → 1
│       │   │   │   │   └── Sim → 1
│       │   │   │   └── Sim → 0
│       │   │   └── Sim → 0
│       │   └──